In [1]:
from SimpleTask import SimpleGridTask
import numpy as np, numpy.random as npr, random as r, SimpleTask
from TransportTask import TransportTask
from NavTask import NavigationTask
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import time
from SeqData import SeqData


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
def _onenHotToEuc(batch_x,batch_y,env):
    oriens_vec={'N':[1,0],'S':[-1,0],'E':[0,1],'W':[0,-1]}
    oriens = ['N', 'E', 'S', 'W'] # 0,1,2,3\n",

    batch_x=list(batch_x)
    batch_y=list(batch_y)

    #print(batch_x[:2])\n",
    for as_init,s_fin,i in zip(batch_x,batch_y,range(len(batch_x))):

        onehot_as_init =np.array(env.deconcatenateOneHotStateVector(as_init))

        int_as_init=[env._oneHotToInt(onehot_as_init[i]) for i in range(0,len(onehot_as_init))]\
                       +[list(as_init[-10:])]
        #int_as_init[2]=oriens_vec[oriens[int_as_init[2]]]\n",
        int_as_init[2]=list(onehot_as_init[2])

        onehot_s_fin=np.array(env.deconcatenateOneHotStateVector(s_fin))

        int_s_fin=[env._oneHotToInt(onehot_s_fin[i]) for i in range(0,len(onehot_s_fin))]\
                       +[list(s_fin[-10:])]

        #int_s_fin[2]=oriens_vec[oriens[int_s_fin[2]]]\n",
        int_s_fin[2]=list(onehot_s_fin[2])
        int_as_init=int_as_init[:2]+[env._oneHotToInt(int_as_init[2])]+int_as_init[3:-1]+int_as_init[-1]
        int_s_fin=int_s_fin[:2]+[env._oneHotToInt(int_s_fin[2])]+int_s_fin[3:-1] #no action space\n",
        
        batch_x[i],batch_y[i]=[np.array(int_as_init),np.array(int_s_fin)]

    return batch_x, batch_y

In [5]:
class ForwardModelDifferentLoss_Euc():
    def __init__(self, 
                obs_space, 
                input_space,
                n_hidden=100
                ):
        self.n_hidden=n_hidden
        self.act_space=input_space-obs_space
        self.obs_space=obs_space
        #Placeholders 
        self.input = tf.placeholder("float", [None, input_space])
        self.truevalue = tf.placeholder("float", [None, obs_space])
        self.pred=self.build_graph(self.input)
        self.saver = tf.train.Saver()
        
        
    def loss_function(self,batch_size,env):
        accTotal=0
        cost=0
        for i in range(0,batch_size):
            predVecs = env.deconcatenateOneHotStateVector(self.pred[i,:],w=1,h=1,no=1)
            labelVecs = env.deconcatenateOneHotStateVector(self.truevalue[i,:],w=1,h=1,no=1)
            for pv,lv,i in zip(predVecs,labelVecs,range(len(predVecs))):
                cost +=  tf.reduce_mean(tf.pow(self.pred - self.truevalue, 2))
                if i!=2:
                    pv=tf.one_hot(tf.cast(pv,tf.int32),env.w)
                    lv=tf.one_hot(tf.cast(lv,tf.int32),env.w)
                
                    pv=tf.reshape(pv,[env.w,])
                    lv=tf.reshape(lv,[env.w,])
                else:
                    pv=tf.one_hot(tf.cast(pv,tf.int32),len(env.oriens))
                    lv=tf.one_hot(tf.cast(lv,tf.int32),len(env.oriens))
                
                    pv=tf.reshape(pv,[len(env.oriens),])
                    lv=tf.reshape(lv,[len(env.oriens),])
                    
                accTotal += tf.cast(tf.equal(tf.argmax(pv,axis=0), tf.argmax(lv,axis=0)), tf.float32)
        
        
        print(cost)
        print(accTotal)
        return cost,accTotal

    
    def build_graph(self,inputVec, reuse=None):
        with tf.variable_scope("forward-model", reuse=reuse):
            hidden = slim.fully_connected(inputVec, self.n_hidden, biases_initializer=None, activation_fn=tf.nn.relu)
            hidden2 = slim.fully_connected(hidden, self.n_hidden, biases_initializer=None, activation_fn=tf.nn.relu)
            return slim.fully_connected(hidden2,self.obs_space, activation_fn=None, biases_initializer=None)
        
    def predict(self, x):
        sess= tf.get_default_session()
        #x.shape = (1,n_steps, n_input)
        return sess.run([self.pred], {self.input:x})

    def load_model(self,model_file_name):
        sess= tf.get_default_session()
        self.saver.restore(sess, model_file_name)

    def train(self,trainset,testset,training_steps,batch_size,env,learning_rate,display_step, model_file_name="FWR_model_"+time.strftime("%Y%m%d-%H%M%S")):
        sess= tf.get_default_session()
        print('Entering loss func')
        cost,accTotal = self.loss_function(batch_size,env)
        print('Defining optimizer')
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
        self.accuracy = accTotal / (batch_size * trainset.env.stateSubVectors) #tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        # Initialize the variables (i.e. assign their default value)
        print('Running TF initializer')
        init = tf.global_variables_initializer()
        sess.run(init)
        noise_sigma = 0.2
        print('Entering train loop')
        for step in range(1, training_steps + 1):
            batch_x, batch_y = trainset.next_batch_nonseq(batch_size)
            batch_x,batch_y= _onenHotToEuc(batch_x,batch_y,env)
            npbx = np.array( batch_x )
            npbxs = npbx.shape
            noise = noise_sigma * np.random.randn( npbxs[0], npbxs[1] )
            batch_x += noise
            sess.run(self.optimizer, feed_dict={self.input: batch_x, self.truevalue: batch_y})
            if step % display_step == 0 or step == 1:
                # Calculate batch accuracy & loss
                acc, loss = sess.run([self.accuracy, cost], feed_dict={self.input: batch_x, self.truevalue: batch_y})
                print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")
        # Calculate accuracy
        test_data, test_label = testset.next_batch_nonseq(5000) 
        test_data,test_label= _onenHotToEuc(batch_x,batch_y,env)
        acc=sess.run(self.accuracy, feed_dict={self.input: test_data, self.truevalue: test_label})
        print("Testing Accuracy:",acc)
        save_path= self.saver.save(sess, "./"+model_file_name+".ckpt")
        print("Model Saved")
        return acc
print('g')

g


In [6]:
print('Reading Data')
s = 'navigation' #'navigation'
trainf, validf = s+"-data-train-small.pickle", s+"-data-test-small.pickle"
train, test   = SeqData(trainf), SeqData(validf)

# classType = NavigationTask if s == 'navigation' else TransportTask
print(train.env.stateSubVectors)
print('Defining Model')
# Parameters
learning_rate = 0.0005
training_steps = 15000 #2000 # 10000
batch_size = 64 #256 #128
display_step = 200
# Network Parameters
n_hidden = 50 #128 #5*train.lenOfInput # hidden layer num of features
len_state = train.lenOfState # linear sequence or not
len_input = train.lenOfInput



print('Initializing FM')
with tf.Graph().as_default(), tf.Session() as sess:
    fm=ForwardModelDifferentLoss_Euc(5,15, n_hidden)
    print('FM initialized')
    fm.train(train,test,training_steps,batch_size,train.env,learning_rate,display_step,"trained_model_3_noise_0_2")




Reading Data
Reading navigation-data-train-small.pickle
	Built
Reading navigation-data-test-small.pickle
	Built
5
Defining Model
Initializing FM
FM initialized
Entering loss func
Tensor("add_638:0", shape=(), dtype=float32)
Tensor("add_639:0", shape=(), dtype=float32)
Defining optimizer
Running TF initializer
Entering train loop
Step 64, Minibatch Loss= 21141.644531, Training Accuracy= 0.10938
Step 12800, Minibatch Loss= 436.631012, Training Accuracy= 0.36875
Step 25600, Minibatch Loss= 274.009399, Training Accuracy= 0.41563
Step 38400, Minibatch Loss= 249.486343, Training Accuracy= 0.42812
Step 51200, Minibatch Loss= 325.420837, Training Accuracy= 0.48750
Step 64000, Minibatch Loss= 298.133972, Training Accuracy= 0.40938
Step 76800, Minibatch Loss= 261.610199, Training Accuracy= 0.49062
Step 89600, Minibatch Loss= 287.184021, Training Accuracy= 0.40938
Step 102400, Minibatch Loss= 258.407898, Training Accuracy= 0.45312
Step 115200, Minibatch Loss= 293.116058, Training Accuracy= 0.4000

ValueError: attempt to get argmax of an empty sequence